In [ ]:
!pip install ../input/timm-0-4-12-py3-none-any-whl/timm-0.4.12-py3-none-any.whl

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import albumentations as A
import torch
import torch.nn.functional as F
from tqdm import tqdm
import sys
import math
import cv2
import time
import timm
from albumentations.augmentations.geometric.functional import bbox_rot90
import matplotlib.pyplot as plt
from torchvision.ops import box_iou

%matplotlib inline

sys.path.append('../input/tensorflow-great-barrier-reef')
sys.path.append('../input/yolov5-lib-ds')
sys.path.append('/kaggle/input/weightedboxesfusion/')

from ensemble_boxes import weighted_boxes_fusion

In [ ]:
!mkdir -p /root/.config/Ultralytics
!cp /kaggle/input/yolov5-font/Arial.ttf /root/.config/Ultralytics/

In [ ]:
import greatbarrierreef
env = greatbarrierreef.make_env()# initialize the environment
iter_test = env.iter_test()      # an iterator which loops over the test set and sample submission

In [ ]:
# hyp

# attention
ATT_ENABLED = True
ATT_ANCHOR_SCORE = 0.15
ATT_BOOST_SCORE = 0.1
ATT_SPLITOR_THRESHOLD = 4

# predict
FINAL_THRESHOLD = 0.08

In [ ]:
splitor = timm.create_model("efficientnet_b0", num_classes=1)
splitor.load_state_dict(torch.load("../input/patric-seq-splitor/splitor.pt"))
splitor.eval()
splitor.cuda()
print("fine")

In [ ]:
def create_scaler(pivot, mode, strenth=2.5, max_weight=None):
    if max_weight is None:
        max_weight = strenth

    def _func(boxes):
        areas = (boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1])
        for i, (area, box) in enumerate(zip(areas, boxes)):
            # boxes[i, 4] *= np.exp(1. - thr / area)
            if mode == "desc":
                boxes[i, 4] *= min(strenth**(1-area/pivot), strenth)
            elif mode == "asc":
                boxes[i, 4] *= min(strenth**(-1+area/pivot), strenth)
            else:
                raise Exception("No such mode")
        return boxes

    return _func

In [ ]:
def filter_by_area(boxes, operator, threshold):
    areas = (boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1])

    if operator == ">":
        return boxes[areas > threshold]
    elif operator == "<":
        return boxes[areas < threshold]
    else:
        raise Exception("No such operator")

In [ ]:
# # visualize
# pivot = 2000.0
# strenth = 2.7
# mode = "asc"
# x = np.linspace(1, 10000, num=10000)
# if mode == "asc":
#     y = np.clip(strenth**(-1+x/pivot), 0, strenth)
# else:
#     y = np.clip(strenth**(1-x/pivot), 0, strenth)

# plt.plot(x, y)

In [ ]:
def wbf(mmdet_list, weights, thres=0.0):

    for mmdet_res in mmdet_list:
        mask = np.ones_like(mmdet_res[:, :4])
        mask[:, 0], mask[:, 1], mask[:, 2], mask[:, 3] = 1280.0, 720.0, 1280.0, 720.0
        mmdet_res[:, :4] /= mask

    boxes_list = [ mmdet_res[:, :4] for mmdet_res in mmdet_list ]
    score_list = [ mmdet_res[:, 4] for mmdet_res in mmdet_list ]
    label_list = [ [0] * len(boxes) for boxes in boxes_list ]

    res = weighted_boxes_fusion(
        boxes_list,
        score_list,
        label_list,
        weights=weights,
        conf_type="avg",
        skip_box_thr=thres
    )

    boxes = res[0]
    scores = res[1]

    mask = np.ones_like(boxes)
    mask[:, 0], mask[:, 1], mask[:, 2], mask[:, 3] = 1280.0, 720.0, 1280.0, 720.0
    boxes *= mask

    return np.concatenate([boxes, scores[:, np.newaxis]], axis=1)

In [ ]:
chk1280 = "../input/patric-final-models/r1280-e13.pt"
chk1800 = "../input/patric-final-models/r1800-e13.pt"
chk2400 = "../input/patric-yolov5l6-multiscale-full-e11/best.pt"
chk2400a = "../input/2400-seeds-2022/best.pt"
chk2400b = "../input/patric-2400-seeds-888/best.pt"
chk2400c = "../input/patric-2400-seeds-777/best.pt"
chk3200 = "../input/3200-full/best.pt"

models = [
#     {
#         "chk": chk2400,
#         "size": 3600,
#         "conf": 0.10,
#         "w": 2
#     },
#     {
#         "chk": chk2400a,
#         "size": 4000,
#         "conf": 0.10,
#         "w": 2,
#         "rotate": "90deg"
#     },
#     {
#         "chk": chk3200,
#         "size": 3200,
#         "conf": 0.10,
#         "w": 2,
#         "rotate": "90deg",
#     },
#     {
#         "chk": chk2400b,
#         "size": 2400,
#         "conf": 0.10,
#         "rotate": "270deg",
#         "w": 2
#     },
#     {
#         "chk": chk3200,
#         "size": 3200,
#         "conf": 0.10,
#         "w": 2,
#         "rotate": "90deg",
#     },
#     {
#         "chk": chk3200,
#         "size": 4000,
#         "conf": 0.10,
#         "w": 2,
#     },
#     {
#         "chk": chk2400,
#         "size": 4800,
#         "conf": 0.10,
#         "rotate": "90deg",
#         "w": 2
#     },

    
    
    # adapt CV part
#     {
#         "chk": chk1280,
#         "size": 1800,
#         "conf": 0.01,
#         "w": 1,
#     },

#     {
#         "chk": chk1800,
#         "size": 1800,
#         "conf": 0.01,
#         "w": 1,
#     },
#     {
#         "chk": chk1800,
#         "size": 2000,
#         "conf": 0.01,
#         "w": 1,
#         "rotate": "90deg",
#     },

    {
        "chk": chk2400,
        "size": 2400,
        "conf": 0.01,
        "w": 1,
    },
    {
        "chk": chk2400,
        "size": 2400,
        "conf": 0.01,
        "w": 1,
        "rotate": "90deg",
    },
#     {
#         "chk": chk2400a,
#         "size": 2800,
#         "conf": 0.01,
#         "w": 1,
#         "rotate": "90deg",
#     },


#     {
#         "chk": chk3200,
#         "size": 3200,
#         "conf": 0.01,
#         "w": 1,
#     },

#     {
#         "chk": chk3200,
#         "size": 3200,
#         "conf": 0.01,
#         "w": 1,
#         "rotate": "90deg",
#     },
#     {
#         "chk": chk3200,
#         "size": 3400,
#         "conf": 0.01,
#         "w": 1,
#     },
#     {
#         "chk": chk3200,
#         "size": 3600,
#         "conf": 0.01,
#         "w": 1,
#         "rotate": "180deg",
#     },

#     {
#         "chk": chk3200,
#         "size": 4000,
#         "conf": 0.01,
#         "w": 1,
#         "rotate": "90deg",
#     },


#     # adapt LB part
#     {
#         "chk": chk2400,
#         "size": 4800,
#         "conf": 0.01,
#         "w": 1,
#         "rotate": "180deg",
#     },

#     {
#         "chk": chk2400b,
#         "size": 4000,
#         "conf": 0.01,
#         "w": 1,
#     },

#     {
#         "chk": chk2400c,
#         "size": 3200,
#         "conf": 0.01,
#         "w": 1,
#         "rotate": "90deg",
#     },
]


In [ ]:
for m in models:
    m["model"] = torch.hub.load('../input/yolov5-lib-ds',
                                'custom',
                                path=m["chk"],
                                source='local',
                                force_reload=True)
    m["model"].conf = 0.01
    m["__attention_area__"] = np.ndarray(shape=[0, 5], dtype=float)

In [ ]:
def create_rotate_back_fn(deg):
    
    h, w = {
        "90deg": (1280, 720),
        "180deg": (720, 1280),
        "270deg": (1280, 720)
    }[deg]

    bbox_rot90_count = {
        "90deg": 1,
        "180deg": 2,
        "270deg": 3
    }[deg]

    norm_back = np.array([1280, 720, 1280, 720, 1])

    def _func(xyxyc):
        xyxyc /= np.array([w, h, w, h, 1])
        xyxyc = np.array([*bbox_rot90(xyxyc, bbox_rot90_count, None, None), xyxyc[4]])
        return xyxyc * norm_back

    return _func


rotates_funcs = {
    "90deg": (cv2.ROTATE_90_CLOCKWISE, create_rotate_back_fn("90deg")),
    "180deg": (cv2.ROTATE_180, create_rotate_back_fn("180deg")),
    "270deg": (cv2.ROTATE_90_COUNTERCLOCKWISE, create_rotate_back_fn("270deg"))
}

att_area = np.ndarray(shape=[0, 5], dtype=float)
previous_image = np.zeros(shape=(720, 1280, 3))

def infer(image):
    global previous_image

    cat_image = np.concatenate([previous_image, image], axis=0)
    previous_image = image.copy()
    res = splitor(torch.tensor(cat_image / 255.0).permute(2, 0, 1).unsqueeze(0).float().cuda()).item()
    is_continouse = res < ATT_SPLITOR_THRESHOLD

    all_boxes = []
    ws = []
    for m in models:
        augment = m["augment"] if "augment" in m else False
        if 'rotate' in m:
            rc = m['rotate']
            cv_flag, rotate_back_fn = rotates_funcs[rc]
            image_rotated = cv2.rotate(image, cv_flag)
            preds_rotated = m["model"](image_rotated, size=m["size"], augment=augment).pandas().xyxy[0]
            preds_rotated = preds_rotated[['xmin', 'ymin', 'xmax', 'ymax', 'confidence']].to_numpy()
            for i, box in enumerate(preds_rotated):
                preds_rotated[i] = rotate_back_fn(box)
            boxes = preds_rotated
        else:
            r = m["model"](image, size=m["size"], augment=augment).pandas().xyxy[0]
            boxes = r[['xmin', 'ymin', 'xmax', 'ymax', 'confidence']].to_numpy()
        if "area_scaler" in m:
            boxes = m["area_scaler"](boxes)
        
        if "area" in m:
            operator, threshold = m["area"]
            boxes = filter_by_area(boxes, operator, threshold)

        if ATT_ENABLED:
            # if not continouse, clear the previous attention area
            if not is_continouse:
                m["__attention_area__"] = np.ndarray(shape=[0, 5], dtype=float)
            boxes = boxes.astype(float)
            _att_area = m["__attention_area__"].copy()
            _boxes = boxes.copy()
            ious = box_iou(torch.tensor(_boxes[:, :4]), torch.tensor(_att_area[:, :4])).numpy()
            if len(ious):
                target_index = ious.argmax(axis=0)
                if len(target_index) > 0:
                    boxes[target_index, 4] += ATT_BOOST_SCORE

            att_area = _boxes[_boxes[:, 4] > ATT_ANCHOR_SCORE]
            m["__attention_area__"] = att_area

        boxes = boxes[boxes[:, 4] > m["conf"]]
        all_boxes.append(boxes)
        ws.append(m["w"])
    preds = wbf(all_boxes, ws)

    # filtering out area <= 500
    # preds = preds[((preds[:, 2] - preds[:, 0]) * (preds[:, 3] - preds[:, 1])) > 500]
    return preds

In [ ]:
df = pd.read_csv("../input/tensorflow-great-barrier-reef/train.csv")
df = df[df.annotations != '[]']
item = df.sample(1).iloc[0]
image = cv2.imread(f"../input/tensorflow-great-barrier-reef/train_images/video_{item.video_id}/{item.video_frame}.jpg")
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

In [ ]:
res = infer(image)
print(res)

In [ ]:
# 22s ~= 9 hours infering
start = time.time()
for i in range(10):
    infer(image)
end = time.time()
hours = round((end - start) / 22 * 9, 1)
print(f"About {hours} hours")

In [ ]:
for idx, (img, pred_df) in enumerate(tqdm(iter_test)):
    anno = ''
    r = infer(img)
    if r.shape[0] == 0:
        anno = ''
    else:
        for b in r:
            if b[4] > FINAL_THRESHOLD:
                anno += '{} {} {} {} {} '.format(b[4], int(b[0]), int(b[1]), int(b[2]-b[0]), int(b[3]-b[1]))

    pred_df['annotations'] = anno.strip(' ')
    env.predict(pred_df)